In [2]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

# To show plots inline in the notebook
%matplotlib inline

### DATA CLEANING OF fact_ad_revenue.csv

In [26]:
file_path = '../Datasets/Raw Datasets/fact_ad_revenue.csv'

df = pd.read_csv(file_path)
df.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments
0,ED1005,A001,2023-Q2,22613.69,EUR,NaN
1,ED1005,A002,Q1-2019,39366.88,USD,NaN
2,ED1001,A003,Q3-2023,3709860.00,INR,NaN
3,ED1003,A002,Q3-2023,40969.55,USD,NaN
4,ED1007,A003,4th Qtr 2020,51779.40,USD,NaN


In [27]:
df.isnull().sum()

edition_id       0
ad_category      0
quarter          0
ad_revenue       0
currency         0
comments       682
dtype: int64

In [5]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   edition_id   720 non-null    object 
 1   ad_category  720 non-null    object 
 2   quarter      720 non-null    object 
 3   ad_revenue   720 non-null    float64
 4   currency     720 non-null    object 
 5   comments     38 non-null     object 
dtypes: float64(1), object(5)
memory usage: 33.9+ KB


Index(['edition_id', 'ad_category', 'quarter', 'ad_revenue', 'currency',
       'comments'],
      dtype='object')

In [28]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

In [7]:
df['quarter'].unique()

array(['2023-Q2', 'Q1-2019', 'Q3-2023', '4th Qtr 2020', '2024-Q2',
       '2019-Q2', 'Q3-2021', '4th Qtr 2023', '2020-Q2', '2022-Q2',
       '2021-Q2', '4th Qtr 2024', 'Q1-2020', 'Q3-2022', 'Q3-2019',
       '4th Qtr 2021', 'Q1-2022', '4th Qtr 2022', 'Q3-2024', 'Q1-2023',
       'Q1-2024', 'Q3-2020', 'Q1-2021', '4th Qtr 2019'], dtype=object)

In [29]:
def standardize_quarter(q):
    q = str(q).strip()

    # Format: YYYY-Qn → Already correct
    if re.match(r'\d{4}-Q[1-4]', q):
        return q

    # Format: Qn-YYYY → Flip it
    match = re.match(r'Q([1-4])-(\d{4})', q, re.IGNORECASE)
    if match:
        return f"{match.group(2)}-Q{match.group(1)}"

    # Format: 4th Qtr 2023, 1st Qtr 2020 etc.
    match = re.match(r'([1-4])(st|nd|rd|th)?\s+Qtr\s+(\d{4})', q, re.IGNORECASE)
    if match:
        return f"{match.group(3)}-Q{match.group(1)}"

    return np.nan  # if unmatched


In [30]:
df['quarter'] = df['quarter'].apply(standardize_quarter)
df.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments
0,ED1005,A001,2023-Q2,22613.69,EUR,NaN
1,ED1005,A002,2019-Q1,39366.88,USD,NaN
2,ED1001,A003,2023-Q3,3709860.00,INR,NaN
3,ED1003,A002,2023-Q3,40969.55,USD,NaN
4,ED1007,A003,2020-Q4,51779.40,USD,NaN


In [31]:
def standardize_currency(c):
    c = str(c).strip()

    if re.match(r'^IN RUPEES$', c, re.IGNORECASE):
        return 'INR'  
    else:
        return c


In [32]:
df['currency'] = df['currency'].apply(standardize_currency)

In [33]:
df.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments
0,ED1005,A001,2023-Q2,22613.69,EUR,NaN
1,ED1005,A002,2019-Q1,39366.88,USD,NaN
2,ED1001,A003,2023-Q3,3709860.00,INR,NaN
3,ED1003,A002,2023-Q3,40969.55,USD,NaN
4,ED1007,A003,2020-Q4,51779.40,USD,NaN


In [34]:
exchange_df = pd.read_csv('../Datasets/Raw Datasets/exchange_rates.csv')
exchange_df.head()

,quarter,USD_to_INR,EUR_to_INR
0,2019-Q1,69.9,78.4
1,2019-Q2,70.2,78.9
2,2019-Q3,71.1,79.5
3,2019-Q4,71.5,80.2
4,2020-Q1,71.8,81.0


In [35]:
df = df.merge(exchange_df, on='quarter', how='left')
df.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments,USD_to_INR,EUR_to_INR
0,ED1005,A001,2023-Q2,22613.69,EUR,NaN,78.5,88.5
1,ED1005,A002,2019-Q1,39366.88,USD,NaN,69.9,78.4
2,ED1001,A003,2023-Q3,3709860.00,INR,NaN,79.0,89.2
3,ED1003,A002,2023-Q3,40969.55,USD,NaN,79.0,89.2
4,ED1007,A003,2020-Q4,51779.40,USD,NaN,74.5,86.0


In [36]:
def convert_to_inr(row):
    if row['currency'] == 'INR':
        return round(row['ad_revenue'],2)
    elif row['currency'] == 'USD':
        return round(row['ad_revenue'] * row['USD_to_INR'],2)
    elif row['currency'] == 'EUR':
        return round(row['ad_revenue'] * row['EUR_to_INR'],2)
    else:
        return np.nan

df['revenue_in_INR'] = df.apply(convert_to_inr, axis=1)


In [37]:
df.head()

,edition_id,ad_category,quarter,ad_revenue,currency,comments,USD_to_INR,EUR_to_INR,revenue_in_INR
0,ED1005,A001,2023-Q2,22613.69,EUR,NaN,78.5,88.5,2001311.56
1,ED1005,A002,2019-Q1,39366.88,USD,NaN,69.9,78.4,2751744.91
2,ED1001,A003,2023-Q3,3709860.00,INR,NaN,79.0,89.2,3709860.00
3,ED1003,A002,2023-Q3,40969.55,USD,NaN,79.0,89.2,3236594.45
4,ED1007,A003,2020-Q4,51779.40,USD,NaN,74.5,86.0,3857565.30


In [38]:
df['year'] = df['quarter'].str[:4]

In [39]:
df.to_csv('../Datasets/Processed Datasets/fact_ad_revenue.csv', index=False)

### DATA CLEANING OF fact_city_readiness.csv

In [19]:
file_path = '../Datasets/Raw Datasets/fact_city_readiness.csv'

df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,city_id,quarter,literacy_rate,smartphone_penetration,internet_penetration
0,0,C001,2019-Q1,89.16,75.76,56.53
1,1,C001,2019-Q2,88.76,76.45,55.97
2,2,C001,2019-Q3,88.83,75.32,56.52
3,3,C001,2019-Q4,89.25,75.83,56.94
4,4,C001,2020-Q1,89.13,75.03,56.45


In [20]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              240 non-null    int64  
 1   city_id                 240 non-null    object 
 2   quarter                 240 non-null    object 
 3   literacy_rate           240 non-null    float64
 4   smartphone_penetration  240 non-null    float64
 5   internet_penetration    240 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 11.4+ KB


Index(['Unnamed: 0', 'city_id', 'quarter', 'literacy_rate',
       'smartphone_penetration', 'internet_penetration'],
      dtype='object')

In [21]:
df = df.drop('s_no', axis=1)

KeyError: "['s_no'] not found in axis"

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

In [ ]:
df['year'] = df['quarter'].str[:4]

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.to_csv('../Datasets/Processed Datasets/fact_city_readiness.csv', index=False)

### DATA CLEANING of fact_digital_pilot.csv

In [ ]:
file_path = '../Datasets/Raw Datasets/fact_digital_pilot.csv'

df = pd.read_csv(file_path)
df.head()

In [ ]:
df.info()
df.columns

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [ ]:
df['launch_month'] = pd.to_datetime(df['launch_month'], format='%Y-%m')

In [ ]:
df['year'] = df['launch_month'].dt.year

In [ ]:
df['month_name'] = df['launch_month'].dt.strftime('%B')

In [ ]:
df['launch_month'] = df['launch_month'].dt.strftime('%Y-%m')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.to_csv('../Datasets/Processed Datasets/fact_digital_pilot.csv', index=False)

### DATA CLEANING OF dim_ad_category.xlsx

In [ ]:
file_path = '../Datasets/Raw Datasets/dim_ad_category.xlsx'

df = pd.read_excel(file_path)
df.head()

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [ ]:
df.info()
df.columns
df.shape

In [ ]:
df.to_csv('../Datasets/Processed Datasets/dim_ad_category.csv', index=False)

### DATA CLEANING OF dim_city.xlsx

In [ ]:
file_path = '../Datasets/Raw Datasets/dim_city.xlsx'

df = pd.read_excel(file_path)
df.head()

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [ ]:
df['state'] = df['state'].str.title()

In [ ]:
df.info()
df.columns
df.shape

In [ ]:
df['city'] = df['city'].str.title()

In [ ]:
df.head(10)

In [ ]:
df.to_csv('../Datasets/Processed Datasets/dim_city.csv', index=False)

### DATA CLEANING OF fact_print_sales.xlsx

In [ ]:
file_path = '../Datasets/Raw Datasets/fact_print_sales.xlsx'

df = pd.read_excel(file_path)
df.head()

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [ ]:
df['language'] = df['language'].str.title()

In [ ]:
df['state'] = df['state'].str.title().str.replace('_', ' ', regex=False)

In [ ]:
df['month'] = pd.to_datetime(df['month'])

In [ ]:
df['year'] = df['month'].dt.year

In [ ]:
df['month'] = df['month'].dt.strftime('%B')

In [ ]:
df['copies_sold'] = df['copies_sold'].astype(str).str.replace('â‚¹', '', regex=False)


In [ ]:
df['copies_sold'] = df['copies_sold'].astype(int)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.to_csv('../Datasets/Processed Datasets/fact_print_sales.csv', index=False)